<Title> Tourist Locations and Historic Sites</Title>
<b>The purpose of this project is to identify tourist office locations for the purposes of taking walking tours in the city of Rome, Italy. The idea is that we want to identify historic sites and position our locations so that they are at the idea locations to those historic sites for walking.</b>
Folium maps often do not show. Please find all maps in the repository

In [1]:
!conda install -c conda-forge folium=0.11.0 --yes

import json
import math
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.11.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-3.1           |   py36h45558ae_0         615 KB  conda-forge
    folium-0.11.0              |             py_0          61 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1    

Using the Foursquare API we enter our credentials which have been removed here for privacy reasons

In [2]:
CLIENT_ID = '' # removed for privacy reasons
CLIENT_SECRET = '' # removed for privacy reasons
VERSION = '20200808'

In [3]:
# url for the GET request

url = 'https://api.foursquare.com/v2/venues/explore?&clie\
nt_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        'Rome, Italy', 
        25000, # searching in radius of 25km
        50, # the 50 most popular historic sites
        '4deefb944765f83613cdba6e') # category ID for "Historic Site"

In [4]:

# GET request

results = requests.get(url).json()["response"]['groups'][0]['items']

In [5]:

historic_sites_list = []

for res in results:
    historic_sites_list = historic_sites_list + [(res['venue']['name'],
                                                  res['venue']['location']['lat'],
                                                  res['venue']['location']['lng'])]

In [6]:

# coordinates

municipi_coord  = pd.DataFrame([('I', 41.893056, 12.482778),
                                ('II', 41.929958, 12.518931),
                                ('III', 41.936081, 12.535117),
                                ('IV', 41.933492, 12.598747),
                                ('V', 41.890664, 12.548489),
                                ('VI', 41.869658, 12.632731),
                                ('VII', 41.8817, 12.5228),
                                ('VIII', 41.841228, 12.484289),
                                ('IX', 41.814878, 12.479981),
                                ('X', 41.847328, 12.280531),
                                ('XI', 41.855283, 12.494761),
                                ('XII', 41.8761, 12.4501),
                                ('XIII', 41.899142, 12.424158),
                                ('XIV', 41.940964, 12.418628),
                                ('XV', 41.955436, 12.48485)],
                               columns = ['Municipio', 'Latitude', 'Longitude'])

In [7]:

# coordinates of Rome

map_Rome = folium.Map(location=[41.883333, 12.5], zoom_start=11)

for lat, lng, mun in zip(municipi_coord['Latitude'], municipi_coord['Longitude'],municipi_coord['Municipio']):
    label = folium.Popup('Municipio {}; coordinates: {}, {}'.format(mun, lat, lng), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Rome)  
    
map_Rome

In [8]:

latitudes = municipi_coord['Latitude']
longitudes = municipi_coord['Longitude']

venues_list=[]
for lat, lng in zip(latitudes, longitudes):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&clie\
nt_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat,
        lng,
        2000, 
        50, # default
        '4deefb944765f83613cdba6e') # category ID for "Historic Site"
            
    # make the GET request
    try:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       
        for res in results:
            historic_sites_list = historic_sites_list + [(res['venue']['name'],
                                                          res['venue']['location']['lat'],
                                                          res['venue']['location']['lng'])]
    except:
        pass

In [9]:

historic_sites = pd.DataFrame(historic_sites_list, columns = ['Historic Site', 'Latitude', 'Longitude'])

In [10]:
historic_sites.head(10)

,Historic Site,Latitude,Longitude
0,Colosseo,41.890633,12.492378
1,Foro Romano,41.892030,12.487037
2,Foro di Cesare,41.894128,12.485232
3,Palatino,41.888234,12.487209
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496
5,Foro di Traiano,41.894729,12.484871
6,Terme di Caracalla,41.878990,12.492443
7,Portico d'Ottavia,41.892382,12.478500
8,Scalinata di Trinità dei Monti,41.905974,12.482647
9,Teatro di Marcello,41.891931,12.479798


In [11]:
print(historic_sites.shape)
historic_sites.drop_duplicates(subset = 'Historic Site', keep = 'first', inplace = True)
print(historic_sites.shape)

(119, 3)
(84, 3)


We find the most populare historic sites

In [12]:


map_historic_sites = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng, site in zip(historic_sites['Latitude'], historic_sites['Longitude'], historic_sites['Historic Site']):
    label = folium.Popup('{}'.format(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_historic_sites)  
    
map_historic_sites

In [13]:

print(historic_sites.shape)
historic_sites = historic_sites[(historic_sites['Historic Site'] != 'Barbiellini Amidei') & \
                                (historic_sites['Historic Site'] != 'Palazzo degli Uffici di EUR Spa')]
print(historic_sites.shape)

(84, 3)
(82, 3)


In [14]:
historic_sites.reset_index(inplace = True, drop = True)
historic_sites.head(10)

,Historic Site,Latitude,Longitude
0,Colosseo,41.890633,12.492378
1,Foro Romano,41.892030,12.487037
2,Foro di Cesare,41.894128,12.485232
3,Palatino,41.888234,12.487209
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496
5,Foro di Traiano,41.894729,12.484871
6,Terme di Caracalla,41.878990,12.492443
7,Portico d'Ottavia,41.892382,12.478500
8,Scalinata di Trinità dei Monti,41.905974,12.482647
9,Teatro di Marcello,41.891931,12.479798


In [15]:


map_historic_sites = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng, site in zip(historic_sites['Latitude'], historic_sites['Longitude'], historic_sites['Historic Site']):
    label = folium.Popup('{}'.format(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_historic_sites)  
    
map_historic_sites

In [16]:

historic_sites['Latitude'] *= math.pi/180
historic_sites['Longitude'] *= math.pi/180

In [17]:
historic_sites['Longitude'] *= math.cos(41.883333*(math.pi/180))
historic_sites.columns = ['Historic Site', 'Latitude (rad)', 'modified Longitude']

In [18]:
historic_sites.head()

,Historic Site,Latitude (rad),modified Longitude
0,Colosseo,0.731129,0.162327
1,Foro Romano,0.731154,0.162258
2,Foro di Cesare,0.731190,0.162234
3,Palatino,0.731088,0.162260
4,Stadio palatino (Stadio palatino | Stadio di D...,0.731076,0.162263


In [19]:

df_clustering = historic_sites.drop('Historic Site', axis = 1) # we remove the column containing strings

kmeans = KMeans(n_clusters = 5, # the number of tourist centers we want to build
                random_state=0).fit(df_clustering)

In [20]:
centers = pd.DataFrame(kmeans.cluster_centers_, columns = ['Latitude', 'Longitude'])


centers['Latitude'] *= 180/math.pi
centers['Longitude'] *= 180/( math.pi * math.cos(41.883333*(math.pi/180)) )

In [21]:
centers

,Latitude,Longitude
0,41.891833,12.487710
1,41.900684,12.472790
2,41.878225,12.548237
3,41.930894,12.521352
4,41.865195,12.507724


In [22]:
historic_sites_clustered = historic_sites.copy(deep = True)
historic_sites_clustered.columns = ['Historic Site', 'Latitude', 'Longitude']

# adding a column with the cluster labels
historic_sites_clustered['Cluster'] = kmeans.labels_

# transforming the coordinates back to the old values
historic_sites_clustered['Latitude'] *= 180/math.pi
historic_sites_clustered['Longitude'] *= 180/( math.pi * math.cos(41.883333*(math.pi/180)) )

In [23]:

historic_sites_clustered.head(3)

,Historic Site,Latitude,Longitude,Cluster
0,Colosseo,41.890633,12.492378,0
1,Foro Romano,41.892030,12.487037,0
2,Foro di Cesare,41.894128,12.485232,0


In [24]:
map_complete = folium.Map(location=[41.89, 12.5], zoom_start=13, tiles = 'CartoDB positron')

colors = ['#8000ff', '#00b5eb', 'brown', '#ffb360', 'green']

for lat, lng, site, cluster in zip(historic_sites_clustered['Latitude'],
                                   historic_sites_clustered['Longitude'],
                                   historic_sites_clustered['Historic Site'],
                                   historic_sites_clustered['Cluster']):
    label = folium.Popup(str(site), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=colors[int(cluster)],
        fill=True,
        fill_color=colors[int(cluster)],
        fill_opacity=0.7).add_to(map_complete)   
    
for lat, lng in zip(centers['Latitude'], centers['Longitude']):
    label = folium.Popup('Tourist Office', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5.5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_complete)

map_complete

In [25]:
R = 6371000 # radius of earth, in meters

def dist_points(lat_1, lng_1, lat_2, lng_2):
    
   
    lat_1 *= math.pi / 180
    lat_2 *= math.pi / 180
    lng_1 *= ( math.pi * math.cos(41.883333*(math.pi/180)) ) / 180
    lng_2 *= ( math.pi * math.cos(41.883333*(math.pi/180)) ) / 180
    
    
    distance = R * math.sqrt( (lat_2 - lat_1) ** 2 + (lng_2 - lng_1) ** 2 )
    
    return distance

In [26]:

def min_cluster_dist(lat, lng):
    distances = []
    
 
    for i in range(5):
        distances = distances + [dist_points(lat,
                                             lng,
                                             centers['Latitude'][i],
                                             centers['Longitude'][i])]
    
    return min(distances) # distance to the closest center(s)

In [27]:
historic_sites_clustered['Distance to nearest Tourist Office (in meters)'] = \
historic_sites_clustered.apply(lambda row : min_cluster_dist(row['Latitude'],
                                                             row['Longitude']), axis=1)

In [28]:
historic_sites_clustered.head(10)

,Historic Site,Latitude,Longitude,Cluster,Distance to nearest Tourist Office (in meters)
0,Colosseo,41.890633,12.492378,0,408.806963
1,Foro Romano,41.892030,12.487037,0,59.831513
2,Foro di Cesare,41.894128,12.485232,0,327.375581
3,Palatino,41.888234,12.487209,0,402.413801
4,Stadio palatino (Stadio palatino | Stadio di D...,41.887582,12.487496,0,473.026931
5,Foro di Traiano,41.894729,12.484871,0,398.610420
6,Terme di Caracalla,41.878990,12.492443,0,1480.886973
7,Portico d'Ottavia,41.892382,12.478500,0,764.931537
8,Scalinata di Trinità dei Monti,41.905974,12.482647,1,1005.930716
9,Teatro di Marcello,41.891931,12.479798,0,655.092169


In [29]:

historic_sites_clustered['Distance to nearest Tourist Office (in meters)'].max()

2053.040674488882

In [30]:


map_centers = folium.Map(location=[41.883333, 12.5], zoom_start=12)

for lat, lng in zip(centers['Latitude'], centers['Longitude']):
    label = folium.Popup('Tourist Office', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=1,
        parse_html=False).add_to(map_centers)  
    
map_centers

In [31]:
centers

,Latitude,Longitude
0,41.891833,12.487710
1,41.900684,12.472790
2,41.878225,12.548237
3,41.930894,12.521352
4,41.865195,12.507724


In [ ]:
<Title> Folium maps don not often show on Github. All images are located in the repository</Title>
As a conclusion to this research we find 5 locations that are ideal for the tourist offices that have a good amount of historic sites nearby with a few miles of walking that is ideal for visitors to the city of Rome.
